In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/Users/liaopeng3/logs/google_analytics_all/train.csv')

In [ ]:
from pandas.io.json import json_normalize

In [ ]:
import json

In [5]:
col_select = ['device','geoNetwork','totals','trafficSource']

In [6]:
df3 = pd.read_csv('/Users/liaopeng3/logs/google_analytics_all/train.csv',\
                  converters={cols:json.loads for cols in col_select},dtype={'fullVisitorId': 'str'},header=0)

In [7]:
for cols in col_select:
    column_as_df = json_normalize(df3[cols])
    column_as_df.columns = [f"{cols}.{subcolumn}" for subcolumn in column_as_df.columns]
    df3 = df3.drop(cols,axis=1)
    df3 = df3.merge(column_as_df,left_index=True,right_index=True)

In [106]:
df3=pd.read_csv('/Users/liaopeng3/logs/google_analytics_all/train_df_3.csv',dtype={'fullVisitorId': 'str'})

In [107]:
df3.head()

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,...,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,1472830385,1,1472830385,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,NaN,(not set),(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,1472880147,1,1472880147,Firefox,desktop,False,...,NaN,NaN,NaN,NaN,NaN,(not set),(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,1472865386,1,1472865386,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,NaN,(not set),(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,1472881213,1,1472881213,UC Browser,desktop,False,...,NaN,NaN,NaN,NaN,NaN,(not set),google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,1472822600,2,1472822600,Chrome,mobile,True,...,NaN,NaN,NaN,NaN,NaN,(not set),(not provided),organic,NaN,google


In [108]:
df3.shape

(903653, 31)

In [109]:
cols_drop = []
for cols in df3.columns:
    if df3[cols].nunique()==1:
        print(cols)
        cols_drop.append(cols)

In [110]:
df3 = df3.drop(cols_drop,axis=1)

# Format the Numerous Variables

In [111]:
list_v = ['totals.hits','totals.pageviews','trafficSource.adwordsClickInfo.page']

In [112]:
for cols in list_v:
    df3[cols]=df3[cols].astype(float)

In [113]:
df3.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'device.browser',
       'device.deviceCategory', 'device.isMobile', 'device.operatingSystem',
       'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.hits', 'totals.pageviews',
       'totals.transactionRevenue', 'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.keyword', 'trafficSource.medium',
       'trafficSource.referralPath', 'trafficSource.source'],
      dtype='object')

In [114]:
df3.dtypes

channelGrouping                                  object
date                                              int64
fullVisitorId                                    object
sessionId                                        object
visitId                                           int64
visitNumber                                       int64
visitStartTime                                    int64
device.browser                                   object
device.deviceCategory                            object
device.isMobile                                    bool
device.operatingSystem                           object
geoNetwork.city                                  object
geoNetwork.continent                             object
geoNetwork.country                               object
geoNetwork.metro                                 object
geoNetwork.networkDomain                         object
geoNetwork.region                                object
geoNetwork.subContinent                         

# Fill Null Cells with Most Frequent Value

In [115]:
for cols in df3.columns:
    if df3[cols].isna().sum()>0:
        val_tmp = df3[cols].mode()[0]
        df3[cols]=df3[cols].fillna(value=val_tmp)
    

# Convert the Object Values to Numeric

In [116]:
for cols in df3.columns:
    if df3[cols].dtypes == 'O':
        df3[cols] = df3[cols].astype('category')
        df3["new_"+cols] = df3[cols].cat.codes
        df3 = df3.drop(cols,axis=1)
        df3 = df3.rename(columns={"new_"+cols:cols})

In [117]:
df3.head()

,date,visitId,visitNumber,visitStartTime,device.isMobile,totals.hits,totals.pageviews,totals.transactionRevenue,trafficSource.adwordsClickInfo.page,channelGrouping,...,geoNetwork.subContinent,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,20160902,1472830385,1,1472830385,False,1.0,1.0,16990000.0,1.0,4,...,21,17,0,10152,1,0,11,5,0,149
1,20160902,1472880147,1,1472880147,False,1.0,1.0,16990000.0,1.0,4,...,1,17,0,10152,1,0,11,5,0,149
2,20160902,1472865386,1,1472865386,False,1.0,1.0,16990000.0,1.0,4,...,19,17,0,10152,1,0,11,5,0,149
3,20160902,1472881213,1,1472881213,False,1.0,1.0,16990000.0,1.0,4,...,16,17,0,10152,1,0,1098,5,0,149
4,20160902,1472822600,2,1472822600,True,1.0,1.0,16990000.0,1.0,4,...,13,17,0,10152,1,0,11,5,0,149


In [126]:
df3.dtypes

date                                              int64
visitId                                           int64
visitNumber                                       int64
visitStartTime                                    int64
totals.hits                                     float64
totals.pageviews                                float64
totals.transactionRevenue                       float64
trafficSource.adwordsClickInfo.page             float64
channelGrouping                                    int8
fullVisitorId                                     int32
sessionId                                         int32
device.browser                                     int8
device.deviceCategory                              int8
device.operatingSystem                             int8
geoNetwork.city                                   int16
geoNetwork.continent                               int8
geoNetwork.country                                int16
geoNetwork.metro                                

# Convert Date to Weekday

In [119]:
df3.head()

,date,visitId,visitNumber,visitStartTime,device.isMobile,totals.hits,totals.pageviews,totals.transactionRevenue,trafficSource.adwordsClickInfo.page,channelGrouping,...,geoNetwork.subContinent,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,20160902,1472830385,1,1472830385,False,1.0,1.0,16990000.0,1.0,4,...,21,17,0,10152,1,0,11,5,0,149
1,20160902,1472880147,1,1472880147,False,1.0,1.0,16990000.0,1.0,4,...,1,17,0,10152,1,0,11,5,0,149
2,20160902,1472865386,1,1472865386,False,1.0,1.0,16990000.0,1.0,4,...,19,17,0,10152,1,0,11,5,0,149
3,20160902,1472881213,1,1472881213,False,1.0,1.0,16990000.0,1.0,4,...,16,17,0,10152,1,0,1098,5,0,149
4,20160902,1472822600,2,1472822600,True,1.0,1.0,16990000.0,1.0,4,...,13,17,0,10152,1,0,11,5,0,149


In [120]:
df3['date'] = pd.to_datetime(df3['date'])



In [121]:
df3['date'] = df3['date'].dt.dayofweek

In [122]:
df3['visitStartTime'] = pd.to_datetime(df3['visitStartTime'],unit='s')

In [123]:
df3['visitStartTime'] = df3.visitStartTime.map( lambda x: pd.to_datetime(x).hour )

# Convert the BOOL Values

In [124]:
df3['device.isMobile'].dtypes

dtype('bool')

In [125]:
for cols in df3.columns:
    if df3[cols].dtypes=='bool':
        df3['new_'+cols]=df3[cols].apply(lambda x: 1 if x else 0)
        df3=df3.drop(cols,axis=1)
        df3 = df3.rename(columns={'new_'+cols:cols})

# Baseline Model

In [127]:
train_x = df3.drop('totals.transactionRevenue',axis=1)
train_y = df3['totals.transactionRevenue']

In [128]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

regr = RandomForestRegressor(max_depth=2, random_state=0,
                             n_estimators=100)
model=regr.fit(train_x, train_y)


/Users/liaopeng3/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [129]:
def calc_train_error(X_train, y_train, model):
    '''returns in-sample error for already fit model.'''
    predictions = model.predict(X_train)
    mse = mean_squared_error(y_train, predictions)
    rmse = np.sqrt(mse)
    return rmse

In [130]:
from sklearn.metrics import mean_squared_error

In [132]:
import numpy as np

In [133]:
calc_train_error(train_x, train_y, model)

50553600.30219388

In [134]:
param = {'num_leaves': 300,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.005,
         "min_child_samples": 20,
         "boosting": "rf",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.8 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 1,
         "verbosity": -1}

In [135]:
import lightgbm as lgb

In [139]:
from sklearn.model_selection import KFold
import time

In [142]:
folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(df3))
predictions = np.zeros(len(df3))
start = time.time()
features = list(train_x.columns)
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_x.values, train_y.values)):
    trn_data = lgb.Dataset(train_x.iloc[trn_idx].values, label=train_y.iloc[trn_idx].values)
    val_data = lgb.Dataset(train_x.iloc[val_idx].values, label=train_y.iloc[val_idx].values)
    
    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(train_x.iloc[val_idx].values, num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(train_x.values, num_iteration=clf.best_iteration) / folds.n_splits

Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 5.18789e+07	valid_1's rmse: 3.43872e+07
Early stopping, best iteration is:
[18]	training's rmse: 5.18942e+07	valid_1's rmse: 3.40772e+07
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 5.05586e+07	valid_1's rmse: 4.26902e+07
Early stopping, best iteration is:
[14]	training's rmse: 5.09699e+07	valid_1's rmse: 4.20939e+07
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 4.78579e+07	valid_1's rmse: 5.37201e+07
Early stopping, best iteration is:
[9]	training's rmse: 4.77979e+07	valid_1's rmse: 5.41162e+07
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 4.90382e+07	valid_1's rmse: 4.92196e+07
Early stopping, best iteration is:
[15]	training's rmse: 4.89572e+07	valid_1's rmse: 4.91616e+07
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 4.36981e+07	valid_1's r